# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [14]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Settings

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [16]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [17]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [37]:
len(df_reviews)

3212

In [18]:
# Höchstes Datum
max_date = df_reviews['date'].max()

# Niedrigstes Datum
min_date = df_reviews['date'].min()

print("Höchstes Datum:", max_date)
print("Niedrigstes Datum:", min_date)


Höchstes Datum: 2023-10-15
Niedrigstes Datum: 2022-10-15


### Filter Sentences

In [19]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [20]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,0c27b81e-07e8-4b85-9436-f8cf6c7d8207
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,e617aa16-a8a5-4b1e-b840-36a33d04bbe4
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,3c599d52-0e00-41a2-b1cb-737a9f9a618b
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,0ae58816-62ba-4e20-a85c-7381926ceb2b
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,73b79620-3d38-4324-9ce0-9165a21b2dd6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,23664095-33b2-4954-9b9e-15be50b1db88
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,a19dd5ff-fbcb-41f1-9c9c-5e7aeb408732
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,0bbd1b32-38f7-4c19-8270-d44f91fbc5d7
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,67b7048f-76e6-490a-8d34-7d543139ae24


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [21]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
21,3,918788541,2005104,0,Nicht zu empfehlen! Zero Dienstleistungsverstä...,2023-09-28,cariocadagema74,"Berlin, Deutschland",Kein Cent!,1.0,berlin,Augustiner am Gendarmenmarkt,de,Dieser Laden macht den Ruhm Berlins als Servic...,10,ed1e1418-6819-4295-b826-c29fcd6dc01b,nl
54,9,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,a4077ceb-2b33-4db2-bd9f-add2c72f375f,af
103,18,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,df1eea9e-453f-431d-8d6e-0ec553c7c9d8,en
111,21,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,d71577a0-bf68-4019-bcec-1f7ea85bf111,id
115,22,883285244,947082,0,Ganz gut,2023-03-23,Mawenabert,NaN,Ansonsten alles gut,4.0,berlin,Burgermeister,de,Hatte einen Hausmeister Burger mit Pommes Baco...,2,27782d46-368e-4567-a00f-9b9467cbfa4b,fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13318,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Service super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,2,36ee3fe5-c5f2-4784-b96f-af2f0553be23,en
13319,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Essen super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,3,d0c7dd37-98c4-49a7-b0c3-b87c33af9d50,ca
13322,3180,873342513,13144674,5,Wie immer super,2022-12-30,nelala79,NaN,Klar!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Salat mit Genüsenuggets gegessen. Wie man es h...,6,311d6655-82e4-478d-ae75-59e260f35cff,tr
13356,3195,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,Ein tolles Restaurant!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,0,76c69ada-a819-41f4-bda3-7698e046a759,fr


In [22]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

In [38]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Bayrisches Bier mitten in LOC.Das kann man im ...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,0,0c27b81e-07e8-4b85-9436-f8cf6c7d8207,de
1,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,"Das Lokal ist immer voll, eine Reservierung is...",5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,1,e617aa16-a8a5-4b1e-b840-36a33d04bbe4,de
2,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Innen ist es gemütlich aber auch laut.,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,2,3c599d52-0e00-41a2-b1cb-737a9f9a618b,de
3,0,921348897,2005104,0,Bayrisches Bier mitten in Berlin,2023-10-15,wolfganga300,Erlangen,Ein Teil des Personals versucht sich südbayeri...,5.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisches Bier mitten in Berlin.Das kann man ...,3,0ae58816-62ba-4e20-a85c-7381926ceb2b,de
4,1,921129385,2005104,0,Denn Sie wissen nicht was Sie tun!!!,2023-10-13,Maximilian-62,"Freiburg, Deutschland",Bayrisch Essen und Gastfreundschaft geht anders.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Bayrisch Essen und Gastfreundschaft geht ander...,0,73b79620-3d38-4324-9ce0-9165a21b2dd6,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Haxe wo man noch nicht mal ein Messer braucht.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,1,23664095-33b2-4954-9b9e-15be50b1db88,de
13422,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Und ich weiß dass ich niemals irgendwo anders ...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,2,a19dd5ff-fbcb-41f1-9c9c-5e7aeb408732,de
13423,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Super nettes aufmerksames Personal.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,3,0bbd1b32-38f7-4c19-8270-d44f91fbc5d7,de
13424,3211,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Chef kommt auch selber an den Tisch und ist se...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...,4,67b7048f-76e6-490a-8d34-7d543139ae24,de


### Balancing

In [23]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(800, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [24]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [25]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,864582525,2,1
1,864598719,0,1
2,864598719,1,1
3,864598719,2,1
4,864598719,3,1
...,...,...,...
3995,921404419,3,1
3996,921404419,6,1
3997,921404419,7,1
3998,921407021,0,1


### Alternate examples based on rating

In [26]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
3.0    800
5.0    800
2.0    800
4.0    800
Name: count, dtype: int64

In [27]:
dfs = [df_balanced_reviews_sentences[df_balanced_reviews_sentences["rating"] == i] for i in range(1, 6)]
df_alternating = pd.DataFrame(columns=df_balanced_reviews_sentences.columns)

max_rows = max(len(df) for df in dfs)
for i in range(max_rows):
    for k in range(5):
        if i < len(dfs[k]):
            df_alternating = pd.concat([df_alternating, dfs[k].iloc[[i]]])

df_balanced_reviews_sentences = df_alternating.reset_index(drop=True)

In [28]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,901,868199195,1190751,13,Unfassbar schlechter Service. Macht einen Boge...,2022-11-12,829ingos,"Frankfurt am Main, Deutschland","Nun kann es schon mal vorkommen, dass der Tisc...",1.0,frankfurt,Trattoria i Siciliani,de,Unfassbar unfreundlicher Service. Obwohl wir r...,2,3c1091d4-b0f7-49cd-a35a-6b1c7b97a3f4,de
1,2721,877172997,2481134,3,3/10,2023-02-02,niclass313,NaN,"Die Cabonara ist mit Sahne, steht zwar so auch...",2.0,mannheim,Da Vino,de,"Wie schon 2 Leute vor mir geschrieben haben, h...",2,11d42ecd-7ec6-4dcc-9e63-2fceae67755b,de
2,2111,920346204,1341713,0,geht so - gibt in der näheren Umgebung bessere...,2023-10-08,Fredionline,"München, Deutschland",Das Augustiner ist natürlich super.,3.0,nürnberg,Gaststätte Nürnberger Bratwurst Glöckl am Dom,de,Das Augustiner ist natürlich super. Die Bayeri...,0,7bc78a70-643c-4e45-a253-4cb580078465,de
3,365,892202081,10322027,11,Ein Plus für die Atmosphäre.,2023-05-27,G8212NHjuliak,"Paris, Frankreich",Der Rest aber sehr toll und die Atmosphäre auc...,4.0,hamburg,Ola Lisboa,de,"Wuselig, lecker und voll. 7/10 fürs Essen. Mus...",3,eb4a1ea1-5e72-4a1e-92de-e9874ca64d43,de
4,83,906408250,1483463,2,"Klasse Abend, vielen Dank!!!",2023-07-29,Thomas B,"Aachen, Deutschland","Diejenigen, die die RESTAURANT_NAME schlecht b...",5.0,hamburg,Bullerei,de,Wir waren zu viert mit unserem Boxer Hund. Uns...,4,04684f97-5d65-4031-8d6c-047039c7c65d,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,921,883130438,6847690,0,Massivst überbewertet,2023-03-22,Comillano,"Frankfurt am Main, Deutschland",Das Lokal hat eine neue Location in der LOC je...,1.0,frankfurt,Thai & Turf - Steakhouse,de,Totale Abzocke und massivst überbewertet. Das ...,1,548432c9-a1fb-4f42-8b99-f7a0de548076,de
3996,523,921079498,742684,0,"Massenabfertigung, sehr unfreundliches Personal",2023-10-13,ingridsO7368HH,NaN,"Sehr unfreundliches Personal, Massenabfertigun...",2.0,münchen,Hofbräuhaus,de,"Sehr unfreundliches Personal, Massenabfertigun...",0,79a74f41-b446-44bb-9519-3051ff0d97f4,de
3997,76,912017295,1483463,1,Leider dieses Mal etwas enttäuschend.....,2023-08-21,ElkeHamburg,"Waldshut-Tiengen, Deutschland","Schweinchenrosa gekachelt, Schlachthausatmosph...",3.0,hamburg,Bullerei,de,Wir waren zum 3. Mal in der Bullerei innerhalb...,7,85c08d45-1b4d-4b04-80a9-6fc271b11601,de
3998,755,881229391,966524,1,"Sehr gut, aber überteuert!",2023-03-06,nordicocean,"Essen, Deutschland","Die Speisen, das Kaffe-Angebot sind sehr wertig.",4.0,köln,Cafe Reichard,de,"Die Speisen, das Kaffe-Angebot sind sehr werti...",0,438e24a9-c326-4831-99b8-aba2f04edf9a,de


### Store as .csv 

In [29]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [30]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [31]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,901,868199195,1190751,13,Unfassbar schlechter Service. Macht einen Boge...,2022-11-12,829ingos,"Frankfurt am Main, Deutschland","Nun kann es schon mal vorkommen, dass der Tisc...",1.0,frankfurt,Trattoria i Siciliani,de,Unfassbar unfreundlicher Service. Obwohl wir r...,2,3c1091d4-b0f7-49cd-a35a-6b1c7b97a3f4,de
1,2721,877172997,2481134,3,3/10,2023-02-02,niclass313,NaN,"Die Cabonara ist mit Sahne, steht zwar so auch...",2.0,mannheim,Da Vino,de,"Wie schon 2 Leute vor mir geschrieben haben, h...",2,11d42ecd-7ec6-4dcc-9e63-2fceae67755b,de
2,2111,920346204,1341713,0,geht so - gibt in der näheren Umgebung bessere...,2023-10-08,Fredionline,"München, Deutschland",Das Augustiner ist natürlich super.,3.0,nürnberg,Gaststätte Nürnberger Bratwurst Glöckl am Dom,de,Das Augustiner ist natürlich super. Die Bayeri...,0,7bc78a70-643c-4e45-a253-4cb580078465,de
3,365,892202081,10322027,11,Ein Plus für die Atmosphäre.,2023-05-27,G8212NHjuliak,"Paris, Frankreich",Der Rest aber sehr toll und die Atmosphäre auc...,4.0,hamburg,Ola Lisboa,de,"Wuselig, lecker und voll. 7/10 fürs Essen. Mus...",3,eb4a1ea1-5e72-4a1e-92de-e9874ca64d43,de
4,83,906408250,1483463,2,"Klasse Abend, vielen Dank!!!",2023-07-29,Thomas B,"Aachen, Deutschland","Diejenigen, die die RESTAURANT_NAME schlecht b...",5.0,hamburg,Bullerei,de,Wir waren zu viert mit unserem Boxer Hund. Uns...,4,04684f97-5d65-4031-8d6c-047039c7c65d,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,921,883130438,6847690,0,Massivst überbewertet,2023-03-22,Comillano,"Frankfurt am Main, Deutschland",Das Lokal hat eine neue Location in der LOC je...,1.0,frankfurt,Thai & Turf - Steakhouse,de,Totale Abzocke und massivst überbewertet. Das ...,1,548432c9-a1fb-4f42-8b99-f7a0de548076,de
3996,523,921079498,742684,0,"Massenabfertigung, sehr unfreundliches Personal",2023-10-13,ingridsO7368HH,NaN,"Sehr unfreundliches Personal, Massenabfertigun...",2.0,münchen,Hofbräuhaus,de,"Sehr unfreundliches Personal, Massenabfertigun...",0,79a74f41-b446-44bb-9519-3051ff0d97f4,de
3997,76,912017295,1483463,1,Leider dieses Mal etwas enttäuschend.....,2023-08-21,ElkeHamburg,"Waldshut-Tiengen, Deutschland","Schweinchenrosa gekachelt, Schlachthausatmosph...",3.0,hamburg,Bullerei,de,Wir waren zum 3. Mal in der Bullerei innerhalb...,7,85c08d45-1b4d-4b04-80a9-6fc271b11601,de
3998,755,881229391,966524,1,"Sehr gut, aber überteuert!",2023-03-06,nordicocean,"Essen, Deutschland","Die Speisen, das Kaffe-Angebot sind sehr wertig.",4.0,köln,Cafe Reichard,de,"Die Speisen, das Kaffe-Angebot sind sehr werti...",0,438e24a9-c326-4831-99b8-aba2f04edf9a,de


In [32]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
2.0    800
3.0    800
4.0    800
5.0    800
Name: count, dtype: int64